In [1]:
import pandas as pd
import numpy as np
import folium
import json
import re

## Filter Function
def mask(df, key, value):
    return df[df[key] == value]
pd.DataFrame.mask = mask

ImportError: No module named pyspark.sql.functions

In [5]:
import findspark
findspark.init()
import pyspark

sc = pyspark.SparkContext()
#f = sc.textFile('Data/domain_location0.csv')

In [7]:
from pyspark.sql.functions import udf


In [9]:
df

DataFrame[SourceCommonName: string, Locations: string, V2Tone: string]

In [ ]:
#####################################################
###########Input Tone, Location Raw Data#############


tone = pd.read_csv('Data/domain_location0.csv',index_col=False,low_memory=False)

for i in range(1,14):
    path = 'Data/domain_location' + str(i) + '.csv'
    temp = pd.read_csv(path,index_col=False,low_memory=False)
    frames = [tone,temp]
    tone = pd.concat(frames)

sourceCountry = pd.read_csv('Data/sourceCountry.csv', index_col = False, low_memory = False)
countryInfo = pd.read_csv('Data/countryInfo.csv', index_col = False, low_memory = False)
countryGeo = pd.read_csv('Data/countrygeo.csv',index_col = False,low_memory = False)
countryGeo.columns = ['fips','lat','lon','country']



In [17]:
countryInfo.head(20)

,iso,iso3,iso_numeric,fips,country,capital,area,population,continent,tld,currency_code,currency_name,phone,postal_code_format,postal_code_regex,languages,geonameid,neighbours,equivalent_fips_code
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565.0,"ES,FR",NaN
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,4975593,AS,.ae,AED,Dirham,971,NaN,NaN,"ar-AE,fa,en,hi,ur",290557.0,"SA,OM",NaN
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,29121286,AS,.af,AFN,Afghani,93,NaN,NaN,"fa-AF,ps,uz-AF,tk",1149361.0,"TM,CN,IR,TJ,PK,UZ",NaN
3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,86754,NaN,.ag,XCD,Dollar,+1-268,NaN,NaN,en-AG,3576396.0,NaN,NaN
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NaN,.ai,XCD,Dollar,+1-264,NaN,NaN,en-AI,3573511.0,NaN,NaN
5,AL,ALB,8,AL,Albania,Tirana,28748.0,2986952,EU,.al,ALL,Lek,355,NaN,NaN,"sq,el",783754.0,"MK,GR,ME,RS,XK",NaN
6,AM,ARM,51,AM,Armenia,Yerevan,29800.0,2968000,AS,.am,AMD,Dram,374,######,^(\d{6})$,hy,174982.0,"GE,IR,AZ,TR",NaN
7,AO,AGO,24,AO,Angola,Luanda,1246700.0,13068161,AF,.ao,AOA,Kwanza,244,NaN,NaN,pt-AO,3351879.0,"CD,NA,ZM,CG",NaN
8,AQ,ATA,10,AY,Antarctica,NaN,14000000.0,0,AN,.aq,NaN,NaN,NaN,NaN,NaN,NaN,6697173.0,NaN,NaN
9,AR,ARG,32,AR,Argentina,Buenos Aires,2766890.0,41343201,SA,.ar,ARS,Peso,54,@####@@@,^([A-Z]\d{4}[A-Z]{3})$,"es-AR,en,it,de,fr,gn",3865483.0,"CL,BO,UY,PY,BR",NaN


In [3]:
######################################################
#################Drop Null Data#######################

tone = tone.dropna()

######################################################
################Rename Data Columns###################

tone.columns = ['Domain','Location','Tone']
tone.reset_index(inplace=True)
tone = tone[['Domain','Location','Tone']]

######################################################
###############Clean the Tone Data####################

pattern = re.compile("^\s+|\s*,\s*|\s+$")
tone['Tone'] = tone['Tone'].apply(lambda x:float(pattern.split(x)[0]))

In [5]:
#####################################################
#########Merge Domain Country to Tone Data###########

tone = pd.merge(tone,sourceCountry,on = ['Domain'],how = 'left')
tone = tone.dropna()
#data.CountryName.value_counts()[:20]

In [10]:
#####################################################
############Chinese Media Data#######################

china = tone.mask('CountryName','China')
us= tone.mask('CountryName','United States')
uk = tone.mask('CountryName','United Kingdom')

In [27]:
#####################################################################################
############Overall Average tone of news from China, U.S. U.K.#######################

print 'The Average Tone of news from China is: '+ str(china.Tone.mean())+'\n'
print 'The Average Tone of news from U.S. is: '+ str(us.Tone.mean())+'\n'
print 'The Average Tone of news from U.K. is: '+ str(uk.Tone.mean())

The Average Tone of news from China is: 0.071357543785

The Average Tone of news from U.S. is: -1.29759147787

The Average Tone of news from U.K. is: -0.776671015338


In [12]:
###########################################################
#######Clean the Location data mentioned in the news#######

def cleanTargetCountry(Loc,Tone):
    count={}
    result = {}
    for i in Loc.split(';'):
        key = i.split('#')[2][:2]
        if  key not in count.keys():
            count[key] = 1
        else:
            count[key] += 1
    totalCount = float(sum(count.values()))
    for i in Loc.split(';'):
        key = i.split('#')[2][:2]
        result[key] = count[key] / totalCount * Tone
        
    #print count
    
    return result


china['Location1'] = china.apply(lambda x: cleanTargetCountry(x['Location'],x['Tone']), axis = 1)
us['Location1'] = us.apply(lambda x: cleanTargetCountry(x['Location'],x['Tone']), axis = 1)
uk['Location1'] = uk.apply(lambda x: cleanTargetCountry(x['Location'],x['Tone']), axis = 1)

/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

In [13]:
###########################################################
##################Initial Result Dataframe#################

## China
china_result = countryInfo[['fips','country']]
china_result['tone'] = 0
china_result['nb_article'] = 0
china_result = china_result.set_index('fips')

## U.S.
us_result = countryInfo[['fips','country']]
us_result['tone'] = 0
us_result['nb_article'] = 0
us_result = us_result.set_index('fips')

## U.K
uk_result = countryInfo[['fips','country']]
uk_result['tone'] = 0
uk_result['nb_article'] = 0
uk_result = uk_result.set_index('fips')


###########################################################
###########Calculate Number of Articles for each target####
#############Maintain the tone result to result df#########

for index, row in china.iterrows():
    target = row['Location1']
    for country in target.keys():
        try:
            china_result.ix[country,'tone'] += target[country]
            china_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass
        
for index, row in us.iterrows():
    target = row['Location1']
    for country in target.keys():
        try:
            us_result.ix[country,'tone'] += target[country]
            us_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass
        
for index, row in uk.iterrows():
    target = row['Location1']
    for country in target.keys():
        try:
            uk_result.ix[country,'tone'] += target[country]
            uk_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass
        


/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/i

In [18]:
######Calculate Average tone for other countries for China, US, UK

china_result['tone'] = china_result['tone']/china_result['nb_article']
us_result['tone'] = us_result['tone']/us_result['nb_article']
uk_result['tone'] = uk_result['tone']/uk_result['nb_article']


In [19]:
us_result.sort(columns = 'tone',ascending=False)[:20]

/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,country,tone,nb_article
fips_x,,,
IM,Isle of Man,0.790325,3
PC,Pitcairn,0.654447,6
SH,Saint Helena,0.311713,4
WS,Samoa,0.235256,75
SE,Seychelles,0.172042,50
CW,Cook Islands,0.159961,8
BT,Bhutan,0.133837,83
TK,Turks and Caicos Islands,0.087136,8
MH,Montserrat,0.078099,22


In [50]:
###########################################################
##################Filter Data##############################

## China

china_result = china_result.dropna()
china_result = china_result.sort(columns = 'nb_article',ascending=False)[:200]
china_result = pd.merge(china_result,countryGeo, on = ['country'],how = 'left')
china_result = china_result.dropna()
china_result = china_result[china_result['nb_article']>1000]

## US
us_result = us_result.dropna()
us_result = us_result.sort(columns = 'nb_article',ascending=False)[:200]
us_result = pd.merge(us_result,countryGeo, on = ['country'],how = 'left')
us_result = us_result.dropna()
us_result = us_result[us_result['nb_article']>1000]

## UK
uk_result = uk_result.dropna()
uk_result = uk_result.sort(columns = 'nb_article',ascending=False)[:200]
uk_result = pd.merge(uk_result,countryGeo, on = ['country'],how = 'left')
uk_result = uk_result.dropna()
uk_result = uk_result[uk_result['nb_article']>1000]

/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:27: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [62]:
china_Map = folium.Map(location=[35.85, 104.19], zoom_start=1)
us_Map = folium.Map(location=[35.85, 104.19], zoom_start=1)
uk_Map = folium.Map(location=[35.85, 104.19], zoom_start=1)

In [68]:
import folium

'''
for index, row in test2.iterrows():
    print index
    Location = [row['lat'],row['lon']]
    name = row['country']
    R = abs(row.tone)
    if row.tone>0:
        Color = '#3186cc'
    else:
        Color = '#cc3131'
    if name == 'Turkey':
        print name
    if index>30:
        folium.CircleMarker(location=Location, radius=1000000*R,
                            popup=name,
                            color=Color,
                            fill_color=Color).add_to(Map)

'''
def draw(Map,loc,r,name):
    if r > 0.05:
        Color = '#3186cc'
    elif r < -0.05:
        Color = '#cc3131'
    else:
        Color = '#0ed148'#0ed148
    
    folium.CircleMarker(location=loc, radius=700000*abs(r),
                            popup=name+': '+str(r),
                            color=Color,
                            fill_color=Color).add_to(Map)
    
china_result.apply(lambda x: draw(china_Map,[x['lat'],x['lon']],x['tone'],x['country']),axis = 1)
us_result.apply(lambda x: draw(us_Map,[x['lat_x'],x['lon_x']],x['tone'],x['country']),axis = 1)
uk_result.apply(lambda x: draw(uk_Map,[x['lat'],x['lon']],x['tone'],x['country']),axis = 1)

        

china_Map

In [64]:
us_Map

In [65]:
uk_Map

In [ ]:
china_Map.save('iconTest_china.html')
us_Map.save('iconTest_us.html')
uk_Map.save('iconTest_uk.html')